In [1]:
import requests 
from bs4 import BeautifulSoup
import re
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver import Chrome

In [2]:
data=[]                                                    # создаём пустой список, в который будут собираться объявления

for i in range(1, 2):                                     # задаем цикл с номерами страниц для загрузки
    browser = webdriver.Chrome()
    print(i)                                               # выводим на печать номер обрабатываемой страницы
    url = f'https://www.list.am/ru/category/56/{i}?type=1&n=1&_a39=2&_a3_1=60&_a3_2=100' # задаем url
    browser.get(url)
    sleep(7)
    soup = BeautifulSoup(browser.page_source, 'lxml')     # Преобразуем дынные для дальнейшей работы
    
    flats = soup.findAll('div', class_='dl')[1].find('div', class_='gl').findAll('a') # находим все объявления на странице
    
    for flat in flats:                                      # проходим по списку с объявлениями и извлекаем информацию о каждом
        
        try:
            
            link = 'https://www.list.am' + flat.get('href')  # извлекаем ссылку на объявление
            price = flat.find('div', class_='p').text        # извлекаем цену на квартиру
            information = flat.find('div', class_='at').text # извлекаем короткое описание квартиры
           
            result = re.findall(r'\d{1,3}', information) # Создаем промежуточную переменную с площадью и кол-ом комнат квартиры
            
            number = re.findall(r'\d+', link)[0]              # извлекаем номер объявления
            region = re.findall(r'\w+,', information)[0][:-1] # извлекаем регион
            rooms = int(result[0])                            # извлекаем количество комнат
            square = int(result[1])                           # извлекаем площадь
            value = re.findall(r'[\֏\$]', price)[0]           # извлекаем валюту цены
            cost = int(re.findall(r'\d+[,]?\d+[,]?\d+', price)[0].replace(',','')) # извлекаем цену
            
            if value == '$':                                  # если цена в долларах, конвертируем её в драммы
                cost *= 390
                value = '֏'

            data.append([number, region, cost, value, rooms, square, price+', ' +information, link]) # добавляем всю информацию

        except:
            print("Исключение было обработано") 
    browser.quit()

print('THE END')

1
Исключение было обработано
THE END


In [3]:
headers = ['number', 'region', 'cost', 'value', 'rooms', 'square', 'information', 'link'] # задаем названия колонок
df = pd.DataFrame(data, columns=headers)                                                  # создаем DataFrame
df = df.drop_duplicates()

In [4]:
df.to_csv(r'D:\Ela\flats_Yerevan\flats_yerevan.csv', index=False, sep=';', encoding='utf8') # сохраняем в файл

In [5]:
df.head(2)

,number,region,cost,value,rooms,square,information,link
0,20597918,Кентрон,468000,֏,2,60,"$1,200 в месяц, Кентрон, 2 ком., 60 кв.м., 9/1...",https://www.list.am/ru/item/20597918
1,15229365,Арабкир,468000,֏,3,100,"$1,200 в месяц, Арабкир, 3 ком., 100 кв.м., 4/...",https://www.list.am/ru/item/15229365
